In [1]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from utils import (get_dates, 
                   get_files, 
                   extract_mta_data,
                   resample_counts,
                   get_entries_exits,
                  )

In [2]:
def anomaly_replacement(df):
    """detects and replaces count anomalies"""
    # make min_samples > 4 and change eps accordingly
    df_out = pd.DataFrame()
    for x in df.columns:
        df[[x]] = df[[x]].fillna(method='bfill').fillna(method='ffill')
        _df = df[[x]]
        
        dbscan = DBSCAN(eps=70, min_samples=4)
        dbscan.fit(_df)
        outliers = np.argwhere(dbscan.labels_ == -1).flatten()
        _df.iloc[outliers,:] = np.nan
        _df = _df.fillna(method='bfill').fillna(method='ffill')
        df_out[x] = _df
    return df_out

In [ ]:
def get_entries_exits(df):
    """Returns dfs for entires and exits by station"""
    df['ent'] = df[['ENTRIES',
                    'STATION',
                    'UNIT',
                    'SCP',
                    'C/A']].groupby(['STATION','UNIT','SCP','C/A']).diff()#.abs()
    df['ex'] = df[['EXITS',
                   'STATION',
                   'UNIT',
                   'SCP',
                   'C/A']].groupby(['STATION','UNIT','SCP','C/A']).diff()#.abs()
    df['ent'].loc[df['ent']< -100] = df['ENTRIES'].loc[df['ent']< -100]
    df['ex'].loc[df['ex']< -100] = df['EXITS'].loc[df['ex']< -100]
    df['ent'] = df['ent'].abs()
    df['ex'] = df['ex'].abs()
    df = df[['TIME','STATION','ent','ex']]
    
    #consider moving anomaly detection to here, prior to resample
    
    df1 = df[['TIME','STATION','ent','ex']].groupby(['STATION','TIME'])\
                                          .sum()\
                                          .reset_index()\
                                          .groupby('STATION')\
                                          .apply(resample_counts)
    df1 = df1.reset_index()
    
    df_out_ent = df1.pivot(index='TIME',
                           columns='STATION',
                           values='ent')

    df_out_ex = df1.pivot(index='TIME',
                          columns='STATION',
                          values='ex')
    return df_out_ent, df_out_ex

In [13]:
def get_entries_exits2(df):
    """Returns dfs for entires and exits by station"""
    df['ent'] = df[['ENTRIES',
                    'STATION',
                    'UNIT',
                    'SCP',
                    'C/A']].groupby(['STATION','UNIT','SCP','C/A']).diff()#.abs()
    df['ex'] = df[['EXITS',
                   'STATION',
                   'UNIT',
                   'SCP',
                   'C/A']].groupby(['STATION','UNIT','SCP','C/A']).diff()#.abs()
    #df['ent'].loc[df['ent']< -100] = df['ENTRIES'].loc[df['ent']< -100]
    #df['ex'].loc[df['ex']< -100] = df['EXITS'].loc[df['ex']< -100]
    df['ent'].loc[df['ent']< 0] = np.nan
    df['ex'].loc[df['ex']< 0] = np.nan
    #df['ent'] = df['ent'].abs()
    #df['ex'] = df['ex'].abs()
    df = df[['TIME','STATION','ent','ex']]
    return df

In [14]:
start_date = "220108"
weeks = 1

df = extract_mta_data(start_date, weeks=weeks)
df.shape

/Users/twitter/Desktop/Current Research/nyc_subway/v2/utils.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_)


(209629, 11)

In [15]:
df1 = get_entries_exits2(df)
df1.shape

/var/folders/td/4jttxk_j59ngw4ck3xdp25th0000gn/T/ipykernel_64605/3312138972.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ent'].loc[df['ent']< 0] = np.nan
/var/folders/td/4jttxk_j59ngw4ck3xdp25th0000gn/T/ipykernel_64605/3312138972.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ex'].loc[df['ex']< 0] = np.nan


(209629, 4)

In [16]:
df1['ent'].max()

481921.0

In [17]:
df1['ent'].min()

0.0

In [ ]:
df2 = df1.groupby(['TIME','STATION']).sum().reset_index().sort_values(['STATION','TIME']).reset_index(drop=True)

In [ ]:
df2['ent'][df2['STATION']=='125 ST']

In [ ]:
from sklearn.cluster import DBSCAN

def anomaly_replacement(df,eps):
    df_out = df.copy()
    #for x in df.columns:
    for x in ['ent','ex']:
        df[[x]] = df[[x]].fillna(method='bfill').fillna(method='ffill')
        _df = df[[x]]
        
        dbscan = DBSCAN(eps=eps, min_samples=3)
        dbscan.fit(_df)
        outliers = np.argwhere(dbscan.labels_ == -1).flatten()
        _df.iloc[outliers,:] = np.nan
        _df = _df.fillna(method='bfill').fillna(method='ffill')
        df_out[x] = _df
    return df_out

In [ ]:
df3 = df2[df2['STATION']=='125 ST'].copy()
df3.head()

In [ ]:
df4 = anomaly_replacement(df3,eps=70)

In [ ]:
df4

In [ ]:
df3[['ent','ex']].plot()

In [ ]:
df4 = anomaly_replacement(df3,eps=70)
df4.plot()
plt.show()

In [ ]:
df2.pivot(index='TIME',
          columns='STATION',
          values='ent')